#  Test 1 {#sec-test_1}
The circuit in @fig-test_1 is from @Johnson1978, Figure 4.8. The circuit has five nodes and nine branches. There are two independent voltage sources, V1 and V2. The value of V2 is 0 volts and the current through V2 is needed for F1, the current controlled current source. There is one independent current source, I1. E1 is a voltage controlled voltage source. The circuit contains four resistors. 

![test circuit](test_1.png){#fig-test_1}

The net list for @fig-test_1 was generated by LTSpice and show below:    
```
* test_1.asc
R2 1 2 1
R3 4 3 1
R4 2 5 2
V1 1 0 2
I1 4 0 9
V2 0 5 0
E1 3 0 1 4 2
F1 2 3 V2 2
R1 1 4 2
.op
.backanno
.end
```

In [79]:
import os
from sympy import *
import numpy as np
from tabulate import tabulate
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
import SymMNA
from IPython.display import display, Markdown, Math, Latex
init_printing()

## Load the net list

In [80]:
net_list = '''
R2 1 2 1
R3 4 3 1
R4 2 5 2
V1 1 0 2
I1 4 0 9
V2 0 5 0
E1 3 0 1 4 2
F1 2 3 V2 2
R1 1 4 2
'''

## Call the symbolic modified nodal analysis function

In [81]:
report, network_df, i_unk_df, A, X, Z = SymMNA.smna(net_list)

Display the equations

In [82]:
# reform X and Z into Matrix type for printing
Xp = Matrix(X)
Zp = Matrix(Z)
temp = ''
for i in range(len(X)):
    temp += '${:s}$<br>'.format(latex(Eq((A*Xp)[i:i+1][0],Zp[i])))

Markdown(temp)

$I_{V1} + v_{1} \cdot \left(\frac{1}{R_{2}} + \frac{1}{R_{1}}\right) - \frac{v_{2}}{R_{2}} - \frac{v_{4}}{R_{1}} = 0$<br>$I_{F1} + v_{2} \cdot \left(\frac{1}{R_{4}} + \frac{1}{R_{2}}\right) - \frac{v_{5}}{R_{4}} - \frac{v_{1}}{R_{2}} = 0$<br>$I_{Ea1} - I_{F1} + \frac{v_{3}}{R_{3}} - \frac{v_{4}}{R_{3}} = 0$<br>$v_{4} \cdot \left(\frac{1}{R_{3}} + \frac{1}{R_{1}}\right) - \frac{v_{3}}{R_{3}} - \frac{v_{1}}{R_{1}} = - I_{1}$<br>$- I_{V2} - \frac{v_{2}}{R_{4}} + \frac{v_{5}}{R_{4}} = 0$<br>$v_{1} = V_{1}$<br>$- v_{5} = V_{2}$<br>$- ea_{1} v_{1} + ea_{1} v_{4} + v_{3} = 0$<br>$I_{F1} - I_{V2} f_{1} = 0$<br>

### Netlist statistics

In [83]:
print(report)

Net list report
number of lines in netlist: 9
number of branches: 9
number of nodes: 5
number of unknown currents: 4
number of RLC (passive components): 4
number of inductors: 0
number of independent voltage sources: 2
number of independent current sources: 1
number of op amps: 0
number of E - VCVS: 1
number of G - VCCS: 0
number of F - CCCS: 1
number of H - CCVS: 0
number of K - Coupled inductors: 0



### Connectivity Matrix

In [84]:
A

⎡1    1     -1            -1                       ⎤
⎢── + ──    ───     0     ───     0   1   0   0  0 ⎥
⎢R₂   R₁     R₂            R₁                      ⎥
⎢                                                  ⎥
⎢  -1     1    1                 -1                ⎥
⎢  ───    ── + ──   0      0     ───  0   0   0  1 ⎥
⎢   R₂    R₄   R₂                 R₄               ⎥
⎢                                                  ⎥
⎢                  1      -1                       ⎥
⎢   0        0     ──     ───     0   0   0   1  -1⎥
⎢                  R₃      R₃                      ⎥
⎢                                                  ⎥
⎢  -1              -1   1    1                     ⎥
⎢  ───       0     ───  ── + ──   0   0   0   0  0 ⎥
⎢   R₁              R₃  R₃   R₁                    ⎥
⎢                                                  ⎥
⎢           -1                   1                 ⎥
⎢   0       ───     0      0     ──   0  -1   0  0 ⎥
⎢            R₄                  R₄           

### Unknown voltages and currents

In [85]:
X

### Known voltages and currents

In [86]:
Z

### Network dataframe

In [87]:
network_df

,element,p node,n node,cp node,cn node,Vout,value,Vname,Lname1,Lname2
0,V1,1,0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
1,V2,0,5,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,R2,1,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN
3,R3,4,3,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,R4,2,5,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,I1,4,0,NaN,NaN,NaN,9.0,NaN,NaN,NaN
6,Ea1,3,0,1,4,NaN,2.0,NaN,NaN,NaN
7,F1,2,3,NaN,NaN,NaN,2.0,V2,NaN,NaN
8,R1,1,4,NaN,NaN,NaN,2.0,NaN,NaN,NaN


### Unknown current dataframe

In [88]:
i_unk_df

,element,p node,n node
0,V1,1,0
1,V2,0,5
2,Ea1,3,0
3,F1,2,3


### Build the network equation matrix

In [89]:
# Put matricies into SymPy 
X = Matrix(X)
Z = Matrix(Z)

NE_sym = Eq(A*X,Z)
NE_sym

⎡          ⎛1    1 ⎞   v₂   v₄⎤        
⎢I_V1 + v₁⋅⎜── + ──⎟ - ── - ──⎥        
⎢          ⎝R₂   R₁⎠   R₂   R₁⎥        
⎢                             ⎥        
⎢          ⎛1    1 ⎞   v₅   v₁⎥        
⎢I_F1 + v₂⋅⎜── + ──⎟ - ── - ──⎥   ⎡ 0 ⎤
⎢          ⎝R₄   R₂⎠   R₄   R₂⎥   ⎢   ⎥
⎢                             ⎥   ⎢ 0 ⎥
⎢                  v₃   v₄    ⎥   ⎢   ⎥
⎢   I_Ea1 - I_F1 + ── - ──    ⎥   ⎢ 0 ⎥
⎢                  R₃   R₃    ⎥   ⎢   ⎥
⎢                             ⎥   ⎢-I₁⎥
⎢      ⎛1    1 ⎞   v₃   v₁    ⎥   ⎢   ⎥
⎢   v₄⋅⎜── + ──⎟ - ── - ──    ⎥ = ⎢ 0 ⎥
⎢      ⎝R₃   R₁⎠   R₃   R₁    ⎥   ⎢   ⎥
⎢                             ⎥   ⎢V₁ ⎥
⎢               v₂   v₅       ⎥   ⎢   ⎥
⎢       -I_V2 - ── + ──       ⎥   ⎢V₂ ⎥
⎢               R₄   R₄       ⎥   ⎢   ⎥
⎢                             ⎥   ⎢ 0 ⎥
⎢             v₁              ⎥   ⎢   ⎥
⎢                             ⎥   ⎣ 0 ⎦
⎢             -v₅             ⎥        
⎢                             ⎥        
⎢    -ea₁⋅v₁ + ea₁⋅v₄ + v₃    ⎥        


In [90]:
# turn the free symbols into SymPy variables
var(str(NE_sym.free_symbols).replace('{','').replace('}',''))

## Symbolic solution

In [91]:
U_sym = solve(NE_sym,X)

Display the symbolic solution

In [92]:
temp = ''
for i in U_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = \frac{- R_{2} V_{2} f_{1} + R_{2} V_{2} - R_{4} V_{1}}{R_{2} f_{1} - R_{2} - R_{4}}$<br>$v_{3} = \frac{I_{1} R_{1} R_{3} ea_{1} + R_{1} V_{1} ea_{1}}{R_{1} ea_{1} + R_{1} + R_{3}}$<br>$v_{4} = \frac{- I_{1} R_{1} R_{3} + R_{1} V_{1} ea_{1} + R_{3} V_{1}}{R_{1} ea_{1} + R_{1} + R_{3}}$<br>$v_{5} = - V_{2}$<br>$I_{V1} = \frac{- I_{1} R_{2} R_{3} f_{1} + I_{1} R_{2} R_{3} + I_{1} R_{3} R_{4} - R_{1} V_{1} ea_{1} f_{1} + R_{1} V_{1} ea_{1} - R_{1} V_{1} f_{1} + R_{1} V_{1} - R_{1} V_{2} ea_{1} f_{1} + R_{1} V_{2} ea_{1} - R_{1} V_{2} f_{1} + R_{1} V_{2} - R_{2} V_{1} f_{1} + R_{2} V_{1} - R_{3} V_{1} f_{1} + R_{3} V_{1} - R_{3} V_{2} f_{1} + R_{3} V_{2} + R_{4} V_{1}}{R_{1} R_{2} ea_{1} f_{1} - R_{1} R_{2} ea_{1} + R_{1} R_{2} f_{1} - R_{1} R_{2} - R_{1} R_{4} ea_{1} - R_{1} R_{4} + R_{2} R_{3} f_{1} - R_{2} R_{3} - R_{3} R_{4}}$<br>$I_{V2} = \frac{V_{1} + V_{2}}{R_{2} f_{1} - R_{2} - R_{4}}$<br>$I_{Ea1} = \frac{- I_{1} R_{1} R_{2} ea_{1} f_{1} + I_{1} R_{1} R_{2} ea_{1} - I_{1} R_{1} R_{2} f_{1} + I_{1} R_{1} R_{2} + I_{1} R_{1} R_{4} ea_{1} + I_{1} R_{1} R_{4} + R_{1} V_{1} ea_{1} f_{1} + R_{1} V_{1} f_{1} + R_{1} V_{2} ea_{1} f_{1} + R_{1} V_{2} f_{1} + R_{2} V_{1} f_{1} - R_{2} V_{1} + R_{3} V_{1} f_{1} + R_{3} V_{2} f_{1} - R_{4} V_{1}}{R_{1} R_{2} ea_{1} f_{1} - R_{1} R_{2} ea_{1} + R_{1} R_{2} f_{1} - R_{1} R_{2} - R_{1} R_{4} ea_{1} - R_{1} R_{4} + R_{2} R_{3} f_{1} - R_{2} R_{3} - R_{3} R_{4}}$<br>$I_{F1} = \frac{V_{1} f_{1} + V_{2} f_{1}}{R_{2} f_{1} - R_{2} - R_{4}}$<br>

## Construct a dictionary of element values

In [93]:
element_value_keys = []
element_value_values = []

for i in range(len(network_df)):
    if network_df.iloc[i]['element'][0] == 'F' or network_df.iloc[i]['element'][0] == 'E' or network_df.iloc[i]['element'][0] == 'G' or network_df.iloc[i]['element'][0] == 'H':
        element_value_keys.append(var(network_df.iloc[i]['element'].lower()))
        element_value_values.append(network_df.iloc[i]['value'])
    else:
        element_value_keys.append(var(network_df.iloc[i]['element']))
        element_value_values.append(network_df.iloc[i]['value'])

element_values = dict(zip(element_value_keys, element_value_values))

element_values

## Numerical solution

In [94]:
NE = NE_sym.subs(element_values)
NE

⎡I_V1 + 1.5⋅v₁ - v₂ - 0.5⋅v₄⎤   ⎡ 0  ⎤
⎢                           ⎥   ⎢    ⎥
⎢I_F1 - v₁ + 1.5⋅v₂ - 0.5⋅v₅⎥   ⎢ 0  ⎥
⎢                           ⎥   ⎢    ⎥
⎢I_Ea1 - I_F1 + 1.0⋅v₃ - v₄ ⎥   ⎢ 0  ⎥
⎢                           ⎥   ⎢    ⎥
⎢   -0.5⋅v₁ - v₃ + 1.5⋅v₄   ⎥   ⎢-9.0⎥
⎢                           ⎥   ⎢    ⎥
⎢  -I_V2 - 0.5⋅v₂ + 0.5⋅v₅  ⎥ = ⎢ 0  ⎥
⎢                           ⎥   ⎢    ⎥
⎢            v₁             ⎥   ⎢2.0 ⎥
⎢                           ⎥   ⎢    ⎥
⎢            -v₅            ⎥   ⎢ 0  ⎥
⎢                           ⎥   ⎢    ⎥
⎢   -2.0⋅v₁ + v₃ + 2.0⋅v₄   ⎥   ⎢ 0  ⎥
⎢                           ⎥   ⎢    ⎥
⎣      I_F1 - 2.0⋅I_V2      ⎦   ⎣ 0  ⎦

In [95]:
U = solve(NE,X)

Display the numerical solution

Six significat digits are displayed so that results can be compared to LTSpice. 

In [96]:
table_header = ['unknown', 'mag']
table_row = []

for name, value in U.items():
    table_row.append([str(name),float(value)])

print(tabulate(table_row, headers=table_header,colalign = ('left','decimal'),tablefmt="simple",floatfmt=('5s','.6f')))

unknown           mag
---------  ----------
v1           2.000000
v2           4.000000
v3           6.285714
v4          -1.142857
v5           0.000000
I_V1         0.428571
I_V2        -2.000000
I_Ea1      -11.428571
I_F1        -4.000000


The node voltages and current through the sources are solved for. The Sympy generated solution matches the LTSpice results:
```
       --- Operating Point ---

V(1):	 2		voltage
V(2):	 4		voltage
V(4):	 -1.14286	voltage
V(3):	 6.28571	voltage
V(5):	 0		voltage
I(F1):	 -4		device_current
I(I1):	 9		device_current
I(R2):	 -2		device_current
I(R3):	 -7.42857	device_current
I(R4):	 2		device_current
I(R1):	 1.57143	device_current
I(E1):	 -11.4286	device_current
I(V1):	 0.428571	device_current
I(V2):	 -2		device_current
```
The results from LTSpice agree with the SymPy results.